In [7]:
!python --version
!pip install --upgrade pip
!pip install pandas
!pip install geopy
!pip install numpy
!pip install gurobipy

Python 3.9.7
     |████████████████████████████████| 2.1 MB 4.1 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 

In [12]:
import pandas as pd


def load_total_demand():
  yearly_sales = pd.read_csv('data/vw_yearly_sales.csv')
  return yearly_sales


def load_dcs():
  # 2 samples
  dcs= pd.read_csv('data/small_list_of_dcs.csv', encoding= 'unicode_escape', names=['ID', "Name","City","State","Address Line 1","Address Line 2", "Coordinates", "Area m3", "Facility Type", "StateConst"])

  # FULL DATASET
  # dcs = pd.read_csv('list_of_dcs.csv', encoding= 'unicode_escape', names=['ID', "Name","City","State","Address Line 1","Address Line 2", "Coordinates", "Area m3", "Facility Type", "StateConst"])
  #stateconst is a constant used to account for the different prices for the cost of living and rent in various states

  dcs['Area sqft'] = round(dcs['Area m3'] * 10.7639)
  return dcs



def load_dealers():
  # https://www.vw.com/app/dccsearch/vw-us/en/Find%20a%20Volkswagen%20Dealer/+/38.353354499999995/-95.3817145/3/+/+/+/+

  # 2 SAMPLES
  dealers= pd.read_csv('data/small_list_of_dealers.csv',names=['ID', "Name","Address Line 1","Address Line 2", "Website","Phone Number", "Coordinates"])

  # FULL DATASET
  # dealers= pd.read_csv('list_of_dealers.csv',names=['ID', "Name","Address Line 1","Address Line 2", "Website","Phone Number", "Coordinates"])
  return dealers


def load_plants():
  plants= pd.read_csv('data/list_of_vw_plants_and_products.csv', encoding= 'unicode_escape',names=['ID', "City","Country", "Model","Coordinates","Current/Former plant"])

  plants_usa = plants.loc[plants["Country"]=="USA"]

  return plants_usa

def load_dealer_demands():

  def single_sample():

  # 4 samples
    dealers= pd.read_csv('data/small_list_of_demand_data.csv',names=['ID',"Name","Rating","Proportion","na"])
    del dealers[dealers.columns[-1]]
    return dealers

  def full_data():
  # FULL DATASET
    dealers= pd.read_csv('data/list_of_demand_data.csv',names=['ID',"Name","Rating","Proportion","na", "na2", "na3"])
    del dealers[dealers.columns[-1]]
    del dealers[dealers.columns[-1]]
    del dealers[dealers.columns[-1]]
    return dealers

  dealers = single_sample() # either call single_sample or full_data

  return dealers


In [13]:
load_dealer_demands() # used google ratings as a indicator of demand/traffic

FileNotFoundError: [Errno 2] No such file or directory: 'small_list_of_demand_data.csv'

In [9]:
load_total_demand()

FileNotFoundError: [Errno 2] No such file or directory: 'vw_yearly_sales.csv'

In [ ]:
load_dcs()

,ID,Name,City,State,Address Line 1,Address Line 2,Coordinates,Area m3,Facility Type,StateConst,Area sqft
0,0,Volkswagen Parts Distribution Center,Ontario,CA,3095 E Cedar St,"Ontario, CA 91761","34.0376080290585, -117.58641507359258",30456,1,511,327825.0
1,1,Volkswagen Parts Distribution Center,Rocklin,CA,1100 Tinker Rd,"Rocklin, CA 95765, United States","38.81328171265908, -121.30150120229099",18360,1,511,197625.0


In [ ]:
load_dealers()

,ID,Name,Address Line 1,Address Line 2,Website,Phone Number,Coordinates
0,0,Alexandria Volkswagen,107 W Glebe Rd,"Alexandria, VA 22305",https://alexandriavw.com,(703) 684-8888,"38.83578067871878, -77.05799331320739"
1,1,Ancira Volkswagen,6125 Bandera Rd,"San Antonio, TX 78238",https://anciravolkswagen.com,(210) 681-2300,"29.488410637739907, -98.60618875952487"
2,2,Ancira Volkswagen of Laredo,2701 N.E. Bob Bullock Loop,"Laredo, TX 78045",https://vwlaredo.com,(956) 717-2200,"27.612614635541124, -99.47040125767218"
3,3,Andy Mohr Volkswagen of Avon,8791 E Us Highway 36,"Avon, IN 46123",https://www.andymohrvw.com,(317) 978-0296,"39.76232468061578, -86.36526215581948"


In [ ]:
load_plants()

,ID,City,Country,Model,Coordinates,Current/Former plant
25,9,Chattanooga,USA,Atlas,"35.077103086458976, -85.13084428224508",current
26,9,Chattanooga,USA,Cross Sport,"35.077103086458976, -85.13084428224508",current
27,9,Chattanooga,USA,ID4,"35.077103086458976, -85.13084428224508",current
28,9,Chattanooga,USA,Passat,"35.077103086458976, -85.13084428224508",former


In [ ]:
### do location work
import networkx as nx
from geopy.distance import geodesic
import numpy

# ALL DISTANCES ARE IN MILES!!!!
# ALL AREAS ARE IN SQFT!!!
def dc_to_dealer_distances(dcs, dealers): # using geodesic distance, may want to add 15% or something onto distance
  distance_dc_to_dealer = numpy.zeros((len(dcs), len(dealers)))
  for dc_id, dc in dcs.iterrows():
    for dealer_id, dealer in dealers.iterrows():
      point1 = dc["Coordinates"].split(",")
      point2 = dealer["Coordinates"].split(",")
      distance = geodesic(point1, point2).miles
      dc_id = dc["ID"]
      dealer_id = dealer["ID"]
      # print ("distance from dc " + str(dc_id)+ " to dealer "+ str(dealer_id) + " is " + str(distance))
      distance_dc_to_dealer[dc_id][dealer_id] = distance
  distance_dc_to_dealer_df = pd.DataFrame(distance_dc_to_dealer)
  return distance_dc_to_dealer_df


def plant_to_dc_distances(plants, dcs):
  # get coordinates of the 1 usa plant
  plant_coordinates = plants['Coordinates'].iloc[0].split(",")
  distance_plants_to_dcs = [] # since we only have 1 usa plant we can just make everything from there (very convenient)
  for dcs_id, dc in dcs.iterrows():
    point1 = plant_coordinates
    point2 = dc["Coordinates"].split(",")
    distance = geodesic(point1, point2).miles
    dc_id = dc["ID"]
    distance_plants_to_dcs.append(distance)
  return distance_plants_to_dcs

plants = load_plants()
dcs = load_dcs()
dealers = load_dealers()

dcs_to_dealers = dc_to_dealer_distances(dcs, dealers)
plants_to_dcs = plant_to_dc_distances(plants, dcs)


print(plants_to_dcs)
print(dcs_to_dealers)

[1844.0657073866873, 2005.3600623066698]
             0            1            2            3
0  2263.452155  1158.436165  1162.247798  1764.014177
1  2363.262580  1445.165292  1475.686061  1862.087265


In [ ]:
plant_to_dc_distances(plants, dcs)

[1844.0657073866873, 2005.3600623066698]

In [ ]:
## Calculate demands
def get_dealer_demands(year):
  dealers = load_dealers()
  dealer_demands = load_dealer_demands()
  total_demands = load_total_demand() # demand for each specific car type


  def get_demands(year):
    year_demands = total_demands.loc[total_demands['year'] == year]
    year_demands = year_demands[["Atlas","Cross Sport","ID4","Passat"]]
    return year_demands

  year_demands = get_demands(2021)
  year_demands = year_demands.values.tolist()[0]

  # print(year_demands)

  proportion_sum = dealer_demands["Proportion"].sum()

  dealer_demands [["Ratio of sales"]] = None
  dealer_demands["Ratio of sales"] = dealer_demands["Proportion"]/proportion_sum

  # print(dealer_demands)

  # # with the sales proportion, get the general sales for each location

  temp_ratio_col = pd.DataFrame(dealer_demands["Ratio of sales"])

  temp = []
  for x in list(dealer_demands["Ratio of sales"]):
    temp1=[]
    for y in list(year_demands):
      temp1.append(round(x*y))
    temp.append(temp1)
  sales = pd.DataFrame(temp, columns=["Atlas","Cross Sport","ID4","Passat"])
  return sales

In [ ]:
sales = get_dealer_demands(2021)
sales
# the left side are the dealer ids

,Atlas,Cross Sport,ID4,Passat
0,32726,19578,7569,11030
1,20980,12551,4853,7071
2,2972,1778,687,1002
3,15705,9395,3633,5293


In [ ]:
## Insert something here with defining facilities


def calculate_dc_cost():


  def calculate_dc_cost_monthly(id, state, facility_type, areasqft):
  # print (id, facility, state, areasqft)
    cost = 1.5*areasqft #2$ baseline
    insurance = 3850  # https://glcdistribution.com/customer-resources/tools/warehousing-insurance-calculator/ 1 million dollar standard

    employee_salaries = (areasqft/2000)*20*(40*4) # https://www.payscale.com/research/CA/Job=Warehouse_Material_Handler/Hourly_Rate

  # pay 1 dollar per sqft for utility
    utilities = areasqft


  # $600 – $950 per month to lease a $50,000 forklift.
  # https://discord.com/channels/260272353118912522/1093655130076893284/1129635416409133076 1 forklift services 5000 sqft
    forklift_cost =  areasqft/5000  *800


    # https://discord.com/channels/260272353118912522/1093655130076893284/1129639090942398504
    price_it_costs_to_build_warehouse_sqft = 100*areasqft
    property_tax = 6/1000*price_it_costs_to_build_warehouse_sqft


    security = 1000 + (5 * 8 * 28*20) # 1000 fixed cost monitoring + 20$/h * 40h/week * 28 people

    if facility_type == 1:
      ft = 1
    else:
      ft = 1.2

    price = (cost + insurance + employee_salaries + utilities + forklift_cost + security + property_tax)* (state/508)*ft
    # 508 is the price of 100 big macs and state is the price of 100 big macs in that state
    return price


  prices = []

  for i in range(len(dcs.index)):
      moneys = calculate_dc_cost_monthly(float(dcs.loc[i,["ID"]]), float(dcs.loc[i,["StateConst"]]), float(dcs.loc[i,["Facility Type"]]),float(dcs.loc[i,["Area sqft"]] ))
      prices.append(moneys*12)
  return prices
  # print(calculate_dc_cost())


# C_i
def calculate_dc_shipment_capacity():
  dcs = load_dcs()
  dcs = dcs[["Area sqft",'Facility Type']]

# The largest car to consider is the Cross Sport
# It has dimensions: 4,966 mm L x 1,990 mm W x 1,723 mm H
# we can round this up to: 17 ft length, 7 foot wide, 6 foot tall
# add about 2 ft to everything as a buffer
# new measurement is about 18 ft length, 8ft wide, 8 foot tall
# so, we can predict that 2 car takes up 144/2, each car takes 72 sq foot

# from https://discord.com/channels/260272353118912522/1093655130076893284/1131271495004459018,
# we see that 60-80% of DCs are for storage. Let's assume 60%


  vehicle_capacity = dcs["Area sqft"]*0.6 /72


# from https://discord.com/channels/260272353118912522/1093655130076893284/1131266537869820075
# we see that most vehicles stay in the dc a couple of weeks (take 2 weeks)
# we can frame this as: All vehicles in the DCS arrive at time 0, and after 2 weeks all are released

  annual_throughput = round(vehicle_capacity) * (52/2)

  # dcs["Annual Throughput"] = annual_throughput

  # print(annual_throughput)
  return (list(annual_throughput))

# WE NEED TO BE CAREFUL ABOUT TYPE 2 FACILITIES:

print(calculate_dc_shipment_capacity())

[71032.0, 42822.0]


# Important Verbiage

Has most of the answers:

def generate_facility_cost

---
### Generally 2-3$ per square foot
### https://www.warehousingandfulfillment.com/resources/fulfillment-cost-calculator/

1. Rent or Mortgage:
[A 50,000 square foot steel warehouse, in the words of one construction expert, will cost between 52,800 and 312,500 per month in addition to monthly rent, utilities, insurance, and other associated costs. A 100,000 increase in the monthly budget will be made up of the cost of a turnkey warehouse, which will also be included in the estimate.](https://www.hechtgroup.com/the-average-size-of-a-small-warehouse-is-1-000-5-000-square-feet/)


2. Insurance:
- a standard warehouse general liability is 1-2 million of coverage, which is about 100/month
3. Salaries:
```
There is no definitive answer to how many employees per square foot warehouse as it depends on a variety of factors such as the type of products being stored, the level of automation in the warehouse, and the workflows. However, a good rule of thumb is to have one employee for every 1,000-1,500 square feet of warehouse space. This allows for efficient operations and ensures that there is enough staff to handle the volume of work.Every employee in a warehouse is allocated about 200-250 square feet of space. Distribution centers had a median square footage of 240,410 in 2016, 176,600 in 2017, and 305,000 in 2018. If a warehouse is properly designed, it will be more efficient and require less space per employee than if it is poorly designed. The number of warehouse employees varies greatly. Warehouses in general are less than 10,000 square feet in size. Eight-hour shifts pay between $14.80 and $19.90 per hour on average. Approximately 35% of the total warehouse space is estimated to be occupied by this type of warehouse.
```
`https://www.hechtgroup.com/how-many-employees-should-a-warehouse-have-per-square-foot/`

4. Utilities:
- [average utlity cost per square foot commercial property is about 1 dollar/sqft for electricity](https://www.iotacommunications.com/blog/average-utility-cost-per-square-foot-commercial-property/)
5. Equipment:
875$ lease a forklift

6. Licensing and Permits:

7. Maintenance and Repairs:

8. Property Taxes:

9. Security:

10. Technology:




# Back to Code

In [ ]:
## Determining the cost of transportation


## THE FUNCTION OF SHIPPING TRUCKS CAN BE APPROXIMATED BY A PIECEWISE FUNCTION
# https://www.desmos.com/calculator/ipxbog2uyr
# constructed from the data here: https://www.marketwatch.com/guides/car-shipping/truck-shipping/#:~:text=According%20to%20quotes%20we%20received,when%20using%20an%20enclosed%20carrier.

def fixed_cost_shipment(): #T_1
  return 186 # cost to drive 0 miles

def calculate_variable_cost_shipment(mileage): #T_2
  # if mileage < 1135:
  #   return mileage * (551/535)
  # return mileage * (0.43)
  return (0.43)

  # TODO: WE ARE ASSUMING FOR SIMPLICYT SAKE 0.43$/MILE AND ARE NOT CONSIDERING THE OTHER FUNCTION FOR MODELLING RIGHT NOW






In [ ]:
## The modelling?
import gurobipy as gp
from gurobipy import GRB


##LOAD DATA

# plants = load_plants()
# dcs = load_dcs()
# dealers = load_dealers()

# dcs_to_dealers = dc_to_dealer_distances(dcs, dealers)
# plants_to_dcs = plant_to_dc_distances(plants, dcs)
# dealer_demands = get_dealer_demands(2021)
# dc_annual_cost = calculate_dc_cost()





plants = load_plants()
dcs = load_dcs()
dealers = load_dealers()

dcs_to_dealers = dc_to_dealer_distances(dcs, dealers)
plants_to_dcs = plant_to_dc_distances(plants, dcs)
dealer_demands = get_dealer_demands(2021) ## car demand for each dealership
dc_annual_cost = calculate_dc_cost()



# define sets
plants_set = [0] # set S
dcs_set = list(dcs["ID"]) # set I
dealers_set = list(dealers["ID"]) # set J
vehicle_set = [0,1,2,3] # we define 0 to 3, corresponding to Atlas, CrossSport, ID4 and Passat vehicles (set V)
facility_type_set = [0, 1] # set P

In [ ]:
dcs_set

[0, 1]

In [ ]:
# Parameter prep - we change the dataframes to arrays of sorts, which we can use indexes and such in the formulation

d_jv = dealer_demands.values
# d_jv[1][0]

m_ij = dcs_to_dealers.values

# dc 1 to dealer 2 should be 1475.6
# m_ij[1][2]

c_si = plants_to_dcs

L_j = 1 # load factor is 1

T_1 = fixed_cost_shipment()

T_2 = calculate_variable_cost_shipment("dummy_value") # see code comments on why we do this as a temp hack for a preliminary model

f_ip = dc_annual_cost

C_i = calculate_dc_shipment_capacity()

In [ ]:
d_jv

array([[32726, 19578,  7569, 11030],
       [20980, 12551,  4853,  7071],
       [ 2972,  1778,   687,  1002],
       [15705,  9395,  3633,  5293]])

In [ ]:
m_ij

array([[2263.45215475, 1158.43616471, 1162.24779847, 1764.01417746],
       [2363.26257953, 1445.1652925 , 1475.68606054, 1862.08726494]])

In [ ]:
c_si

[1844.0657073866873, 2005.3600623066698]

In [ ]:
L_j

1

In [ ]:
T_1

186

In [ ]:
T_2

0.43

In [ ]:
f_ip

[19560591.129921257, 11922485.01968504]

In [ ]:
C_i

[71032.0, 42822.0]

In [ ]:
m = gp.Model("cflp")

# CREATE THE DECISION VARIABLES HERE
# : Annual shipment of type v∈V vehicles from source s∈S to DC location i∈I
x1_siv = m.addVars(plants_set, dcs_set, vehicle_set, vtype=GRB.INTEGER) #
x2_ijv = m.addVars(dcs_set, dealers_set, vehicle_set, vtype=GRB.INTEGER )
y_ip = m.addVars(dcs_set, facility_type_set, vtype=GRB.BINARY)

In [ ]:
x1_siv

{(0, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 3): <gurobi.Var *Awaiting Model Update*>}

In [ ]:
x2_ijv

{(0, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 0, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 0, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 0, 3): <gurobi.Var *Awaiting Model Update*>,


In [ ]:
y_ip

{(0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 1): <gurobi.Var *Awaiting Model Update*>}

In [ ]:
# TODO: FIX THIS
#  m.setObjective(, gp.GRB.MINIMIZE)

# obj2_sub1 = sum(x2_ijv[i, j, v] + x1_siv[s, i, v] for v in vehicle_set for j in dealers_set for s in plants_set for i in dcs_set)
# obj2 = sum(obj2_sub1 * c_si[i] for s in plants_set for i in dcs_set)

# Need to verify them all with Affleck
obj1 = sum((x2_ijv[i, j, v]/L_j)*(T_1 + T_1*m_ij[i, j]) for v in vehicle_set for j in dealers_set for i in dcs_set)

# Since we only have one plant, c_si should only take in i as an index. We are good.
obj2 = sum(sum(x2_ijv[i, j, v] + x1_siv[s, i, v] for v in vehicle_set for j in dealers_set for s in plants_set) * c_si[i] for i in dcs_set)

obj3 = sum(x1_siv[s, i, v] for v in vehicle_set for s in plants_set for i in dcs_set)

# This works for a small example, but not for a larger example
# Can we just do one type of facility?
obj4 = sum(f_ip[i]*y_ip[i, p] for p in facility_type_set for i in dcs_set)

m.setObjective(obj1 + obj2 + obj3 + obj4, gp.GRB.MINIMIZE)

In [ ]:
# Constraints:

# Constraint 1: Demand at each market area for each vehicle type must be met
m.addConstrs(sum(x2_ijv[i, j, v] for i in dcs_set) == d_jv[j, v] for v in vehicle_set for j in dealers_set)

# Constraint 2: Vehicle flows between plants and DCs must be conserved
# Ask Affleck about summing over i twice
m.addConstrs(sum(x1_siv[s, i, v] for s in plants_set) == sum(x2_ijv[i, j, v] for i in dcs_set) for i in dcs_set for v in vehicle_set for j in dealers_set)

# Constraint 3: Check with Affleck
# Total vehicle flow to each DC must satisfy the minimum and maximum capacity requirements of the type of facility installed.
# Also, if there is no DC installed at a location, there is no shipment to that location.
m.addConstrs(C_i[i]*y_ip[2] <= sum(x2_ijv[i, j, v] for v in vehicle_set for j in dealers_set) <= C_i[i]*y_ip[1] + sum(d_jv[j, v] for v in vehicle_set for j in dealers_set) * y_ip[2] for i in dcs_set)

# Constraint 4: Shipment quantities must be nonnegative
m.addConstrs(x1_siv[s, i, v] >= 0 for s in plants_set for v in vehicle_set for i in dcs_set)
m.addConstrs(x2_ijv[i, j, v] >= 0 for i in dcs_set for v in vehicle_set for j in dealers_set)

# Constraint 5: Ask Affleck


{(0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 2): <gurobi.Constr *Awaiting Model Upd

In [ ]:
m.optimize()
# m.write("out.sol")

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 200 rows, 44 columns and 456 nonzeros
Model fingerprint: 0x7dc7e6b5
Variable types: 0 continuous, 44 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+03, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+02, 3e+04]
Presolve removed 40 rows and 4 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [ ]:

# CONSTRAINTS HERE:

# Demand at each market area for each vehicle type must be met

# for v in vehicle_set:
#   print (v)


# for all j and v
# x2_jiv[(j, v)] for j in dealers_set) == 1 for

for v in vehicle_set:
  for j in dealers_set:


    for i in dcs_set:
      sum_of_i = sum(x2_ijv[i,j,v])

      m.addConstrs(sum_of_i = d_jv[j][v])




# gp.quicksum(x2_ijv[(j,v)]) for i in dcs_set == d_jv[j][v] for j in dealers_set


TypeError: ignored

In [ ]:
# Also, if there is no DC installed at a location, there is no shipment to that location

In [ ]:
# The distribution centers must be selected so that all market areas can be reached within r days. Suppose a truck travels on average 300 miles a day.